In [ ]:
import os
import gc
import cv2
import jax
import json
import pdal
import laspy

import whitebox
import rasterio
import traceback
import subprocess

import numpy as np
import pandas as pd
import jax.numpy as np
import geopandas as gpd

from tqdm import tqdm
from typing import Tuple
from jax import jit, vmap

from shapely.geometry import Point, Polygon
from rasterio.transform import xy
from rasterio.fill import fillnodata
from multiprocessing import Process, Queue, Pool, cpu_count, get_context
from skimage.restoration import denoise_tv_chambolle


jax.config.update("jax_enable_x64", True)

# whitebox.download_wbt(linux_musl=True, reset=True)

wbt = whitebox.WhiteboxTools()
wbt.verbose = False


In [2]:
raw_lidar_dir = "/mnt/data/sv_zh/merged_point_cloud"
ground_lidar_dir = "/mnt/data/sv_zh/ground_lidar"

intensity_tif_dir = "/mnt/data/sv_zh/intensity_tif"
elevation_tif_dir = "/mnt/data/sv_zh/elevation_tif"

os.makedirs(ground_lidar_dir, exist_ok=True)
os.makedirs(intensity_tif_dir, exist_ok=True)
os.makedirs(elevation_tif_dir, exist_ok=True)

In [1]:
import geopandas as gpd
from shapely.geometry import box
import subprocess
import os

# Settings
input_laz_folder = "/mnt/data/sv_zh/point_cloud"
output_folder = "/mnt/data/sv_zh/merged_point_cloud"
os.makedirs(output_folder, exist_ok=True)

gdf = gpd.read_file("/mnt/data/sv_zh/point_cloud.gpkg")

gdf["minx"] = gdf.bounds.minx
gdf["miny"] = gdf.bounds.miny

def snap_to_grid(x, y, size=200):
    snapped_x = (x // size) * size
    snapped_y = (y // size) * size
    return snapped_x, snapped_y

gdf["grid_x"], gdf["grid_y"] = zip(*gdf.apply(lambda row: snap_to_grid(row["minx"], row["miny"]), axis=1))

groups = gdf.groupby(["grid_x", "grid_y"])

In [ ]:
tqdm.pandas()
for (grid_x, grid_y), group in tqdm(groups):
    laz_files = group["filename"].tolist()
    laz_paths = [os.path.join(input_laz_folder, f) for f in laz_files]

    output_filename = f"merged_{grid_x}_{grid_y}.las"
    output_path = os.path.join(output_folder, output_filename)

    # Build PDAL merge pipeline
    inputs = [{"filename": path} for path in laz_paths]
    pipeline = {
        "pipeline": inputs + [{"type": "writers.las", "filename": output_path}]
    }

    # Write temporary JSON pipeline file
    import json
    temp_pipeline_path = "temp_pipeline.json"
    with open(temp_pipeline_path, "w") as f:
        json.dump(pipeline, f)

    # Run the pipeline
    subprocess.run(["pdal", "pipeline", temp_pipeline_path])

    print(f"Merged {len(laz_files)} tiles into {output_path}")

# Clean up temp file
if os.path.exists(temp_pipeline_path):
    os.remove(temp_pipeline_path)


# Generate Ground LiDAR (PDAL CSF)

In [4]:
pipeline_template = {
    "pipeline": [
        {
        "type" : "readers.las",
        "filename" : "input.las"
        },
        {
            "type": "filters.range",
            "limits": "Classification[0:18]"
        },
        {
            "type": "filters.csf"
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        },
        {
        "type" : "writers.las",
        "filename" : "output.las"
        }
    ]
}

laz_files = [f for f in os.listdir(raw_lidar_dir) if f.endswith('.laz')]

for laz_file in tqdm(laz_files):
    input_path = os.path.join(raw_lidar_dir, laz_file)
    output_path = os.path.join(ground_lidar_dir, f"ground_{laz_file.replace('.laz', '.las')}")

    if os.path.exists(output_path):
        continue
    pipeline = pipeline_template
    pipeline["pipeline"][0]['filename'] = input_path
    pipeline["pipeline"][-1]['filename'] = output_path

    pipeline_json = json.dumps(pipeline)
    pipeline_obj = pdal.Pipeline(pipeline_json)
    pipeline_obj.execute()

    # print(f"Processed: {laz_file} -> {output_path}")

100%|██████████| 722/722 [02:17<00:00,  5.24it/s]


In [ ]:
pipeline_template = {
    "pipeline": [
        {
        "type" : "readers.las",
        "filename" : "input.las"
        },
        {
            "type": "filters.range",
            "limits": "Classification[0:18]"
        },
        {
            "type": "filters.csf"
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        },
        {
        "type" : "writers.las",
        "filename" : "output.las"
        }
    ]
}

laz_files = [f for f in os.listdir(raw_lidar_dir) if f.endswith('.laz')]

def process_file(laz_file):
    input_path = os.path.join(raw_lidar_dir, laz_file)
    output_filename = f"ground_{laz_file.replace('.laz', '.las')}"
    output_path = os.path.join(ground_lidar_dir, output_filename)

    if os.path.exists(output_path):
        return f"Skipped: {laz_file}"

    pipeline = json.loads(json.dumps(pipeline_template))  # deep copy
    pipeline["pipeline"][0]['filename'] = input_path
    pipeline["pipeline"][-1]['filename'] = output_path

    try:
        pipeline_obj = pdal.Pipeline(json.dumps(pipeline))
        pipeline_obj.execute()
        return f"Processed: {laz_file}"
    except RuntimeError as e:
        return f"Failed: {laz_file} with error {e}"

if __name__ == "__main__":
    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(pool.imap_unordered(process_file, laz_files), total=len(laz_files)))
    
    for res in results:
        print(res)

In [ ]:
pipeline_template = {
    "pipeline": [
        {
        "type" : "readers.las",
        "filename" : "input.las"
        },
        {
            "type": "filters.range",
            "limits": "Classification[0:18]"
        },
        {
            "type": "filters.csf"
        },
        {
            "type": "filters.range",
            "limits": "Classification[2:2]"
        },
        {
        "type" : "writers.las",
        "filename" : "output.las"
        }
    ]
}

las_files = [f for f in os.listdir(raw_lidar_dir) if f.endswith('.las')]

def process_file(las_file):
    input_path = os.path.join(raw_lidar_dir, las_file)
    output_filename = f"ground_{las_file}"
    output_path = os.path.join(ground_lidar_dir, output_filename)

    if os.path.exists(output_path):
        return f"Skipped: {las_file}"

    pipeline = json.loads(json.dumps(pipeline_template))  # deep copy
    pipeline["pipeline"][0]['filename'] = input_path
    pipeline["pipeline"][-1]['filename'] = output_path

    try:
        pipeline_obj = pdal.Pipeline(json.dumps(pipeline))
        pipeline_obj.execute()
        return f"Processed: {las_file}"
    except RuntimeError as e:
        return f"Failed: {las_file} with error {e}"

if __name__ == "__main__":
    with Pool(processes=3) as pool:
        results = list(tqdm(pool.imap_unordered(process_file, las_files), total=len(las_files)))
    
    for res in results:
        print(res)

In [ ]:
def voxel_density_filter(points, voxel_size=0.5):
    """Filter sparse points using voxel grid."""
    voxel_indices = np.floor(points / voxel_size).astype(np.int32)
    dtype = [('x', np.int32), ('y', np.int32), ('z', np.int32)]
    structured_voxels = np.core.records.fromarrays(voxel_indices.T, dtype=dtype)
    _, inverse_indices, counts = np.unique(structured_voxels, return_inverse=True, return_counts=True)
    min_points = np.quantile(counts, 0.3)
    keep_mask = counts[inverse_indices] >= max(min_points, 150)
    return keep_mask

def remove_sparse(input_path, output_path):
    """Process a single LAS file."""
    las = laspy.read(input_path)
    flag = voxel_density_filter(las.xyz, voxel_size=1.0)
    dense_las = laspy.LasData(las.header)
    dense_las.points = las.points[flag]
    dense_las.write(output_path)

def worker(task_queue, progress_queue):
    """Worker process: Computes but does no I/O."""
    while True:
        task = task_queue.get()
        if task is None:  # Sentinel to stop
            break
        input_path, output_path = task
        try:
            remove_sparse(input_path, output_path)
            progress_queue.put(1)  # Update progress
        except Exception as e:
            print(f"Failed processing {input_path}: {e}")
            progress_queue.put(0)

if __name__ == "__main__":
    input_dir = ground_lidar_dir
    output_dir = ground_lidar_dir
    os.makedirs(output_dir, exist_ok=True)

    # Prepare tasks (input/output paths)
    ground_las = [f for f in os.listdir(input_dir) if f.startswith('ground_') and f.endswith('.las')]
    tasks = []
    for las_file in ground_las:
        input_path = os.path.join(input_dir, las_file)
        output_filename = f"dense_{las_file.lstrip('ground_')}"
        output_path = os.path.join(output_dir, output_filename)
        if not os.path.exists(output_path):
            tasks.append((input_path, output_path))

    # Create queues
    task_queue = Queue()
    progress_queue = Queue()

    # Start worker processes (4 workers)
    num_workers = 8
    workers = []
    for _ in range(num_workers):
        p = Process(target=worker, args=(task_queue, progress_queue))
        p.start()
        workers.append(p)

    # Enqueue tasks (single producer)
    for task in tasks:
        task_queue.put(task)

    # Add sentinels to stop workers
    for _ in range(num_workers):
        task_queue.put(None)

    # Track progress with tqdm
    progress = tqdm(total=len(tasks))
    for _ in range(len(tasks)):
        progress.update(progress_queue.get())  # Blocks until a worker reports progress
    progress.close()

    # Clean up
    for p in workers:
        p.join()

In [ ]:
import os
import subprocess
import json
import geopandas as gpd
from shapely.geometry import Polygon

def las_folder_to_bbox_gpkg(folder_path, output_gpkg):
    polygons = []
    filenames = []
    num_pts = []

    for filename in tqdm(os.listdir(folder_path)):
        if filename.lower().endswith('.las') or filename.lower().endswith('.laz'):
            filepath = os.path.join(folder_path, filename)
            # Get bbox using pdal info
            result = subprocess.run(['pdal', 'info', filepath], capture_output=True, text=True)
            info = json.loads(result.stdout)
            try:
                bounds = info["stats"]['bbox']['native']['bbox']
                points = info["stats"]['statistic'][0]['count']
                num_pts.append(points)
                
                min_x, max_x = bounds['minx'], bounds['maxx']
                min_y, max_y = bounds['miny'], bounds['maxy']
                # We don't use Z here for 2D polygons

                # Create a polygon from bbox corners
                poly = Polygon([
                    (min_x, min_y),
                    (max_x, min_y),
                    (max_x, max_y),
                    (min_x, max_y),
                    (min_x, min_y)  # Close the polygon
                ])

                polygons.append(poly)
                filenames.append(filename)

            except Exception:
                num_pts.append(0)
                polygons.append(None)
                filenames.append(filename)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame({'filename': filenames, 'num_pts': num_pts, 'geometry': polygons}, crs="EPSG:2056")  # You can set your real CRS here!

    # Save to GPKG
    gdf.to_file(output_gpkg, driver="GPKG")

# Example usage:
folder = '/mnt/data/sv_zh/point_cloud'
output_gpkg = '/mnt/data/sv_zh/point_cloud.gpkg'
las_folder_to_bbox_gpkg(folder, output_gpkg)

print("✅ Saved bounding boxes to GPKG successfully.")


# Interpolate raster

In [ ]:
las_files = [f for f in os.listdir(ground_lidar_dir) if f.startswith('ground_') and f.endswith('.las')]

for las_file in tqdm(las_files):
    input_path = os.path.join(ground_lidar_dir, las_file)
    elevation_raster = os.path.join(elevation_tif_dir, f"elevation_{las_file.lstrip('ground_').replace('.las', '.tif')}")
    intensity_raster = os.path.join(intensity_tif_dir, f"intensity_{las_file.lstrip('ground_').replace('.las', '.tif')}")

    if not os.path.exists(intensity_raster):
        wbt.lidar_idw_interpolation(
            input_path,
            intensity_raster,
            parameter="intensity",
            returns="first",
            resolution=0.02,
            radius=0.1,
            weight=2.0
        )

    if not os.path.exists(elevation_raster):
        wbt.lidar_idw_interpolation(
            input_path,
            elevation_raster,
            parameter="elevation",
            returns="first",
            resolution=0.02,
            radius=0.1,
            weight=2.0
        )

In [7]:
las_files = [f for f in os.listdir(ground_lidar_dir) if f.startswith('ground_') and f.endswith('.las')]

def process_las_file(las_file):
    input_path = os.path.join(ground_lidar_dir, las_file)
    elevation_raster = os.path.join(elevation_tif_dir, f"elevation_{las_file.lstrip('ground_').replace('.las', '.tif')}")
    intensity_raster = os.path.join(intensity_tif_dir, f"intensity_{las_file.lstrip('ground_').replace('.las', '.tif')}")

    if not os.path.exists(intensity_raster):
        wbt.lidar_idw_interpolation(
            input_path,
            intensity_raster,
            parameter="intensity",
            returns="first",
            resolution=0.02,
            radius=0.1,
            weight=2.0
        )


    if not os.path.exists(elevation_raster):
        wbt.lidar_idw_interpolation(
            input_path,
            elevation_raster,
            parameter="elevation",
            returns="first",
            resolution=0.02,
            radius=0.1,
            weight=2.0
        )
    
    return None

In [ ]:
with Pool(processes=4) as pool:
    results = list(tqdm(pool.imap_unordered(process_las_file, las_files), total=len(las_files)))

for r in results:
    print(r)

In [5]:
ground_lidar_dir

'/mnt/data/sv_zh/ground_lidar'

In [6]:
wbt.set_working_dir(ground_lidar_dir)
wbt.lidar_idw_interpolation(
    output=intensity_tif_dir,
    parameter="intensity",
    returns="first",
    resolution=0.02,
    radius=0.1,
    weight=2.0
)

0

# 2D GT (Circle detection)

In [ ]:
def preprocess(img, nodata_value):
    """
    Preprocess the image for Hough Circle detection.
    Args:
        img: Input image (2D NumPy array).
        nodata_value: Value representing no data in the image.
    Returns:
        Preprocessed image (2D NumPy array).
    """
    img[img == nodata_value] = np.median(img[img>=0])
    img = np.clip(img, 25000, 45000)
    img = cv2.medianBlur(img, 5)

    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    img = img.astype(np.uint8)
    img = denoise_tv_chambolle(img, weight=0.2, channel_axis=None) * 255
    img = img.astype(np.uint8)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(64,64))
    img = clahe.apply(img)
    return img

In [1]:
from sklearn.decomposition import PCA

def preprocess(img):
    """
    Preprocess the image for Hough Circle detection with contrast enhancement using quantile clipping.
    Args:
        img: Input image (2D NumPy array).
        nodata_value: Value representing no data in the image.
        low_q: Lower quantile for contrast clipping (default 2%).
        high_q: Upper quantile for contrast clipping (default 98%).
    Returns:
        Preprocessed image (2D NumPy array).
    """
    valid_mask = img >= 0
    median_val = np.median(img[valid_mask])
    img[~valid_mask] = median_val

    # Compute quantiles and clip
    img = np.clip(img, 0, 1200)

    img = cv2.medianBlur(img, 5)

    # Normalize to 0–255 range
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    # img = img.astype(np.uint8)
    # img = denoise_tv_chambolle(img, weight=0.2, channel_axis=None) * 255
    img = img.astype(np.uint8)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(64, 64))
    img = clahe.apply(img)
    
    return img


def is_circle_flat(points, epsilon=0.15):
    """
    Fits a plane to 3D points and checks if the points lie approximately on that plane.
    
    Args:
        points: (N, 3) array-like list of 3D points.
        epsilon: Threshold for maximum allowed deviation from the plane.
        
    Returns:
        is_flat (bool): True if all points lie within epsilon of the plane.
        normal (np.ndarray): The normal vector of the fitted plane.
        max_distance (float): Maximum distance from points to the plane.
    """

    # Input validation
    if len(points) < 3:
        return False
    
    points = np.asarray(points, dtype=np.float64)
    centroid = points.mean(axis=0)
    centered = points - centroid

    # Check rank deficiency
    if np.linalg.matrix_rank(centered) < 2:
        return False
    
    try:
        # Regularized SVD approach
        _, _, vh = np.linalg.svd(centered + 1e-10*np.random.randn(*centered.shape), 
                                full_matrices=False)
        normal = vh[-1]
        
        # Safe normalization
        norm = np.linalg.norm(normal)
        if norm < 1e-10:
            return False, None
        
    except Exception:
        # Final fallback to PCA if everything else fails

        pca = PCA(n_components=3)
        pca.fit(points)
        normal = pca.components_[2]
        norm = np.linalg.norm(normal)
        if norm < 1e-10:
            return False, None
        print(f"Using PCA for normal vector.")

    normal /= norm

    # Compute distances to plane
    distances = np.dot(centered, normal)
    max_distance = np.abs(np.max(distances) - np.min(distances))

    return int(max_distance < epsilon), max_distance


def to_gdf(circles, transform, crs, elevation_raster):
    """
    Converts detected circles in image coordinates to a GeoDataFrame with 3D polygons.

    Parameters:
    - circles: List of detected circles with each entry as [x_pix, y_pix, r_pix].
    - transform: Affine transform of the raster.
    - crs: Coordinate Reference System of the raster.
    - elevation: 2D NumPy array representing the elevation raster.

    Returns:
    - GeoDataFrame with 3D circle polygons.
    """
    geometries = []
    data = []

    with rasterio.open(elevation_raster) as src_elevation:
        elevation = src_elevation.read(1)
        elev_nodata = src_elevation.profile['nodata']

    for x_pix, y_pix, r_pix in circles:
        # Create a point and buffer it to get a circle polygon
        # adding 2 pixels to the radius to fully cover the circle
        r_pix = r_pix + 1
        center_pix = Point(x_pix, y_pix)
        circle_pix = center_pix.buffer(r_pix)

        img_coords = np.array(circle_pix.exterior.coords).round().astype(int)
        img_coords[:, 1] = np.clip(img_coords[:, 1], 0, elevation.shape[0] - 1)
        img_coords[:, 0] = np.clip(img_coords[:, 0], 0, elevation.shape[1] - 1)
        z_map = elevation[img_coords[:, 1], img_coords[:, 0]]


        if sum(z_map == elev_nodata) > 0 and sum(z_map != elev_nodata) > 0:
            # Replace nodata values with the median of the valid elevation values
            z_map[z_map == elev_nodata] = np.median(z_map[z_map != elev_nodata])
        
        elif sum(z_map == elev_nodata) == len(z_map):
            # If all values are nodata, skip this circle
            print(f"All elevation values are nodata for circle at ({x_pix}, {y_pix})")
            z_map = np.zeros_like(z_map)

        # Calculate center coordinates in map space
        x_center, y_center = xy(transform, y_pix, x_pix)

        # Pixel size (assuming square pixels)
        pixel_size = (transform.a + abs(transform.e)) / 2.0
        r_map = r_pix * pixel_size

        # Create a point and buffer it to get a circle polygon
        center_point = Point(x_center, y_center)
        circle_polygon = center_point.buffer(r_map)

        # Convert to 3D polygon with Z elevation
        coords_3d = [(x, y, z) for x, y, z in np.concatenate([np.array(circle_polygon.exterior.coords), z_map[:, None]], axis=1)]

        try:
            # Check if the circle is flat
            is_flat, max_distance = is_circle_flat(coords_3d, epsilon=0.15)

        except Exception as e:
            print(f"Error checking flatness: {e}")
            continue
        
        circle_polygon = Polygon(coords_3d)

        # Append to lists
        geometries.append(circle_polygon)
        data.append({
            'x_center': x_center,
            'y_center': y_center,
            'radius_pix': r_pix,
            'radius_map': r_map,
            'z_center': np.median(z_map),
            'is_flat': is_flat,
            'max_distance': max_distance
        })

    return gpd.GeoDataFrame(data, geometry=geometries, crs=crs)


def save_empty_gpkg(output_path, crs="EPSG:2056"):  # Using Swiss CRS as example
    # Create empty GeoDataFrame with correct schema
    empty_gdf = gpd.GeoDataFrame(columns=[
        'x_center', 
        'y_center', 
        'radius_pix', 
        'radius_map', 
        'z_center', 
        'is_flat',
        'max_distance',
        'geometry'
    ], crs=crs)
    
    # Save to file
    empty_gdf.to_file(output_path, driver="GPKG")


def process_file(file_name):
    """
    Process a single file to detect manholes and save results to a GeoPackage.
    Args:
        file_name: Name of the input intensity raster file.
    Returns:
        Number of detected manholes or None if no circles were found.
    """
    
    # Define directories
    intensity_tif_dir = "/mnt/data/sv_zh/intensity_tif"
    elevation_tif_dir = "/mnt/data/sv_zh/elevation_tif"
    gpkg_dir = '/mnt/data/sv_zh/gpkg'

    if os.path.exists(os.path.join(gpkg_dir, file_name.replace('.tif', '.gpkg'))):
        return None  
          
    elevation_raster = os.path.join(elevation_tif_dir, file_name.replace('intensity', 'elevation'))
    intensity_raster = os.path.join(intensity_tif_dir, file_name)

    try:
        # Load intensity first and immediately process/release
        with rasterio.open(intensity_raster) as src_intensity:
            intensity = src_intensity.read(1)
            inten_nodata = src_intensity.profile['nodata']
            transform = src_intensity.transform
            crs = src_intensity.crs
        
        # Process and immediately clear intermediate arrays
        intensity = preprocess(intensity)
        processed_img = intensity.copy()
        del intensity  # Explicitly free memory

        circles = cv2.HoughCircles(
            processed_img,
            cv2.HOUGH_GRADIENT,
            dp=1.0,
            minDist=25,     # minimum distance between circle centers
            param1=80,      # higher threshold for Canny edge detector
            param2=20,      # accumulator threshold (lower is more sensitive)
            minRadius=10,
            maxRadius=25
        )

        if circles is not None:
            # print(f'{len(detected_circles)} manholes detected in {file_name}')
            gdf = to_gdf(circles[0], transform, crs, elevation_raster)
            # return gdf
            gdf.to_file(os.path.join(gpkg_dir, file_name.replace('.tif', '.gpkg')), driver="GPKG")
            return len(circles)
        else:
            # Create empty file when no circles found
            output_path = os.path.join(gpkg_dir, file_name.replace('.tif', '.gpkg'))
            print(f"No circles found in {file_name}. Saving empty GeoPackage.")
            save_empty_gpkg(output_path, crs)
            return None
        
    except MemoryError:
        print(f"Memory error processing {file_name}")
        return None
    except cv2.error as e:
        print(f"OpenCV error in {file_name}: {e}")
        return None
    except rasterio.errors.RasterioError as e:
        print(f"Rasterio error in {file_name}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error in {file_name}: {traceback.format_exc()}")
        return None

    finally:
        # Clean up
        gc.collect()


In [4]:
intensity_tif_dir = "/mnt/data/sv_zh/intensity_tif"
# file_list = [f for f in os.listdir(intensity_tif_dir) if f.startswith('intensity_NE_5_')]
file_list = [f for f in os.listdir(intensity_tif_dir) if f.endswith('.tif')]

with Pool(processes=4) as pool:
    results = list(tqdm(pool.imap_unordered(process_file, file_list), total=len(file_list)))

  0%|          | 0/58 [00:00<?, ?it/s]

 33%|███▎      | 19/58 [01:13<01:51,  2.87s/it]

No circles found in intensity_merged_2682600.0_1247600.0.tif. Saving empty GeoPackage.


100%|██████████| 58/58 [03:12<00:00,  3.32s/it]


In [5]:
gpkg_dir = '/mnt/data/sv_zh/gpkg'
file_name = 'zh_manholes_20.gpkg'
cmd = f'ogrmerge.py -f GPKG -o ../{file_name} -single *.gpkg'

# Change to the directory
os.chdir(gpkg_dir)

# Run the command
subprocess.run(cmd, shell=True, check=True)

gdf_gt = gpd.read_file(f'/mnt/data/sv_zh/{file_name}')
gdf_gt = gdf_gt.set_crs(epsg=2056, allow_override=True)
gdf_gt.to_file(f'/mnt/data/sv_zh/{file_name}', driver='GPKG')
# Transform to EPSG:32632
# gdf_transformed = gdf_gt.to_crs(epsg=32632)
# # Replace 'transformed_file.gpkg' with your desired output file name
# gdf_transformed.to_file('/mnt/data/sv_ne/ne_manholes_flat_25000_20_32632.gpkg', driver='GPKG')

Warning 1: Type and subtype of field definition are not compatible. Resetting to OFSTNone


# Generate OD GT

In [1]:
import os
import gc
import cv2
import jax
import json
import pdal
import laspy

import whitebox
import rasterio
import traceback
import subprocess

import numpy as np
import pandas as pd
import jax.numpy as jnp
import geopandas as gpd

from tqdm import tqdm
from typing import Tuple
from jax import jit, vmap

from shapely.geometry import Point, Polygon
from rasterio.transform import xy
from rasterio.fill import fillnodata
from multiprocessing import Process, Queue, Pool, cpu_count, get_context
from skimage.restoration import denoise_tv_chambolle
jax.config.update("jax_enable_x64", True)
jax.config.update('jax_platform_name', 'cpu')




In [2]:
def ypr2mat(yaw: float, pitch: float, roll: float) -> jnp.ndarray:
    """
    Reproduce Metashape's ypr2mat behavior.
    Args:
        yaw (float): Rotation about Y-axis (degrees).
        pitch (float): Rotation about X-axis (degrees).
        roll (float): Rotation about Z-axis (degrees).
    Returns:
        jnp.ndarray: 3x3 rotation matrix.
    """
    # Convert degrees to radians
    yaw = -jnp.radians(yaw)
    pitch = jnp.radians(pitch)
    roll = jnp.radians(roll)

    # Rotation matrices for each axis
    R_x = jnp.array([
        [1, 0, 0],
        [0, jnp.cos(pitch), -jnp.sin(pitch)],
        [0, jnp.sin(pitch), jnp.cos(pitch)]
    ])

    R_y = jnp.array([
        [jnp.cos(roll), 0, jnp.sin(roll)],
        [0, 1, 0],
        [-jnp.sin(roll), 0, jnp.cos(roll)]
    ])

    R_z = jnp.array([
        [jnp.cos(yaw), -jnp.sin(yaw), 0],
        [jnp.sin(yaw), jnp.cos(yaw), 0],
        [0, 0, 1]
    ])

    # Combine rotations: Yaw → Pitch → Roll
    R = R_z @ R_x @ R_y 
    return R

@jit
def transform_to_camera_crs(x: jnp.ndarray, y: jnp.ndarray, z: jnp.ndarray, camera_meta: dict) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Transforms points from local CRS to camera CRS.
    
    Args:
        x, y, z: 1D arrays (JAX arrays, NumPy arrays, or Pandas Series) of 
                 the x, y, z coordinates of points in local CRS.
        camera_pose: Dictionary containing the camera orientation:
                     {'yaw': yaw, 'pitch': pitch, 'roll': roll}
                     
    Returns:
        A Tuple of three 1D arrays (x', y', z') representing points in camera CRS.
    """

    # Rotation 
    R = ypr2mat(camera_meta['yaw'], camera_meta['pitch'], camera_meta['roll'])  # Camera to local
    R_bore = ypr2mat(0, -90, 0)  # Boresight to camera
    # R_bore = ypr2mat(0.5, -90, - 0.3)  # Boresight to camera
    # R_bore = ypr2mat(0.332734, -89.8243, 0)  # Boresight to camera
    R = R @ R_bore.T @ jnp.diag(jnp.array([1, -1, -1]))
     
    # Define a function for transforming a single point
    # from global coordinates to camera coordinates
    def transform_point(px, py, pz):
        global_point = jnp.array([px, py, pz])
        transformed_point = R.T @ (global_point)
        return transformed_point

    # Vectorize the transformation function
    transform_points = vmap(transform_point)

    # Apply the transformation in parallel
    transformed_points = transform_points(x, y, z)  # Shape: (N, 3)
    
    # Split the transformed points back into separate arrays
    x_cam, y_cam, z_cam = transformed_points.T
    
    return x_cam, y_cam, z_cam

@jit
def spherical_projection(x: jnp.ndarray, y: jnp.ndarray, z: jnp.ndarray, w: int, h: int) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Project 3D LiDAR points to 2D spherical coordinates.

    Args:
        x (jnp.ndarray): X coordinates of the points.
        y (jnp.ndarray): Y coordinates of the points.
        z (jnp.ndarray): Z coordinates of the points.
        w (int): Width of the output image.
        h (int): Height of the output image.

    Returns:
        Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]: 
            u (jnp.ndarray): U coordinates in the image.
            v (jnp.ndarray): V coordinates in the image.
            r (jnp.ndarray): Depth distances of the points.
    """
    # Map azimuth and elevation to image coordinates
    f = w / (2 * jnp.pi) 
    u = 0.5 * w + f * jnp.arctan2(x, z)  
    v = 0.5 * h + f * jnp.arctan2(y, jnp.sqrt(x**2 + z**2)) 

    return u, v

def replace_zeros_with_nearest(z):
    z = np.asarray(z)
    non_zero_indices = np.where(z != 0)[0]

    if len(non_zero_indices) == 0:
        return z  # nothing to replace

    result = z.copy()
    for i in np.where(z == 0)[0]:
        # Find the nearest non-zero index
        nearest_index = non_zero_indices[np.argmin(np.abs(non_zero_indices - i))]
        result[i] = z[nearest_index]

    return result

In [3]:
gdf_gt = gpd.read_file('/mnt/Data/StreetView/data/neuchatel/NE_GT_3D.gpkg', layer='ne_gt_3d')
gdf_gt

,id,geometry
0,4,"POLYGON Z ((2560778.546 1204397.79 432.849, 25..."
1,3,"POLYGON Z ((2560818.338 1204422.75 433.012, 25..."
2,8,"POLYGON Z ((2560781.04 1204415.25 433.16, 2560..."
3,5,"POLYGON Z ((2560888.203 1204466.718 433.742, 2..."
4,2063,"POLYGON Z ((2557971.008 1203983.64 534.931, 25..."
...,...,...
1396,2892,"POLYGON Z ((2527886.832 1195469.928 926.811, 2..."
1397,2893,"POLYGON Z ((2527881.852 1195469.449 926.789, 2..."
1398,2894,"POLYGON Z ((2527871.733 1195469.496 926.76, 25..."
1399,2830,"POLYGON Z ((2529407.513 1195464.325 929.91, 25..."


In [4]:
# nan_flag = []
# for gt_idx, circle in gdf_gt.iterrows():
#     # world_coords = np.array(circle.geometry.geoms[0].exterior.coords)
#     world_coords = np.array(circle.geometry.exterior.coords)
#     x, y, z = world_coords.T

#     if (z == 0).any():
#             # replace zero Z values with the nearest non-zero value in z vector
#             nan_flag.append(gt_idx)

# gdf_gt.iloc[nan_flag]

In [5]:
# for gt_idx, circle in gdf_gt.iterrows():
#     # world_coords = np.array(circle.geometry.geoms[0].exterior.coords)
#     world_coords = np.array(circle.geometry.exterior.coords)
#     x, y, z = world_coords.T

#     if (z == 0).all():
#         # Skip if all Z coordinates are zero
#         print(f"Skipping GT {gt_idx} due to zero Z values.")
#     if (z == 0).any():
#             # replace zero Z values with the nearest non-zero value in z vector
#             z = replace_zeros_with_nearest(z)
#     # update circle.geometry with the new z values
#     new_coords = np.column_stack((x, y, z))
#     new_polygon = Polygon(new_coords)
#     gdf_gt.at[gt_idx, 'geometry'] = new_polygon
# gdf_gt = gdf_gt.set_crs(epsg=2056, allow_override=True)
# gdf_gt.to_file('/mnt/data/sv_ne/NE_GT_3D.gpkg', driver='GPKG', overwrite=True)


In [6]:
camera_df = pd.read_csv('/home/shanci/Downloads/export_steep_ypr.csv')  # Replace with your actual file
# target_files = ['20200408_073125_003815.jpg', '20200408_073125_003816.jpg', '20200410_113822_000050.jpg', '20200408_073125_003817.jpg', '20200408_073125_003818.jpg']
# camera_df = camera_df[[f in target_files for f in camera_df.File.values]]
camera_df['File'] = camera_df['name'] + '.jpg'
camera_df.drop(columns=['name'], inplace=True)
camera_df.rename(columns={'E':'x', 'N': 'y', 'H':'z', 'y': 'course', 'p':'pitch', 'r': 'roll'}, inplace=True)
camera_df

,x,y,z,course,pitch,roll,File
0,2.560767e+06,1.204712e+06,455.581098,105.895672,-7.131229,1.825059,20200408_105231_001931.jpg
1,2.560787e+06,1.204708e+06,454.220432,100.162288,-7.111336,0.608047,20200408_105231_001927.jpg
2,2.560616e+06,1.204657e+06,467.589898,44.622530,-7.111165,2.067972,20200408_105231_001967.jpg
3,2.560590e+06,1.204588e+06,471.643720,39.649812,-7.101109,-1.411226,20200408_105231_001982.jpg
4,2.560758e+06,1.204715e+06,456.224258,107.918263,-7.058721,1.091168,20200408_105231_001933.jpg
...,...,...,...,...,...,...,...
133,2.560638e+06,1.204641e+06,467.074302,207.065909,2.643797,2.774642,20200408_105231_001858.jpg
134,2.560646e+06,1.204659e+06,465.583884,198.606049,2.758709,2.897041,20200408_105231_001862.jpg
135,2.560785e+06,1.204705e+06,454.382229,279.734773,2.799553,-0.137647,20200408_105231_001896.jpg
136,2.560648e+06,1.204664e+06,465.184202,196.669942,2.826690,3.809456,20200408_105231_001863.jpg


In [7]:
# Load camera metadata into a DataFrame
all_camera_df = pd.read_csv('/mnt/Data/ai3d/potree/pointclouds/streetview_ne/pano/coordinates.txt', sep='\t')  # Replace with your actual file
# camera_df = camera_df[(camera_df.x >= 2559490) & (camera_df.x <= 2559600) &
#                       (camera_df.y >= 1203500) & (camera_df.y <= 1203650)]
all_camera_df.set_index('File', inplace=True)
camera_df = all_camera_df.loc[camera_df.File]
camera_df.reset_index(inplace=True)

In [9]:
camera_df

,File,Time,x,y,z,course,pitch,roll
0,20200408_105231_001931.jpg,296001.0,2560767.431,1204712.362,455.557,106.043683,-7.172697,1.816100
1,20200408_105231_001927.jpg,296000.0,2560787.027,1204708.213,454.218,100.318340,-7.172395,0.586676
2,20200408_105231_001967.jpg,296019.0,2560616.239,1204656.888,467.622,44.778868,-7.179832,2.049722
3,20200408_105231_001982.jpg,296028.0,2560590.139,1204588.442,471.724,39.811514,-7.166640,-1.433506
4,20200408_105231_001933.jpg,296002.0,2560757.768,1204715.091,456.216,108.060589,-7.117750,1.105352
...,...,...,...,...,...,...,...,...
133,20200408_105231_001858.jpg,295916.0,2560638.284,1204640.920,467.044,207.215954,2.518321,2.774931
134,20200408_105231_001862.jpg,295919.0,2560646.091,1204659.378,465.553,198.755847,2.648801,2.909677
135,20200408_105231_001896.jpg,295939.0,2560784.770,1204705.462,454.388,279.893321,2.670385,-0.124579
136,20200408_105231_001863.jpg,295920.0,2560647.494,1204664.227,465.163,196.817409,2.712496,3.823511


In [8]:
tqdm.pandas()

height, width = 4000, 8000
image_info_ls = []
annotations = []
annotation_id = 1
# Iterate over each camera
for cam_idx, cam in tqdm(camera_df.iterrows()):
    cam_point = Point(cam.x, cam.y)
    cam_buffer = cam_point.buffer(20)  # 10-meter radius

    # Filter circles within 20 meters
    nearby_circles = gdf_gt[gdf_gt.geometry.centroid.within(cam_buffer)]
    
    # Proceed to project these circles to the image frame
    if len(nearby_circles) == 0:
        continue

    for gt_idx, circle in nearby_circles.iterrows():
        # world_coords = np.array(circle.geometry.geoms[0].exterior.coords)
        world_coords = np.array(circle.geometry.exterior.coords)
        x, y, z = world_coords.T

        if (z == 0).all():
            # Skip if all Z coordinates are zero
            print(f"Skipping GT {gt_idx} at camera {cam_idx} due to zero Z values.")
            continue
        elif (z == 0).any():
            # replace zero Z values with the nearest non-zero value in z vector
            z[z == 0] = np.median(z[z != 0])

        # shift coordinates center to gt  
        x = jnp.array(x - cam.x)
        y = jnp.array(y - cam.y)
        z = jnp.array(z - cam.z)

        cam_ori = {'yaw': cam.course + 0.5, 'pitch': cam.pitch + 0.0, 'roll': cam.roll - 0.3}
        # cam_ori = {'yaw': cam.course, 'pitch': cam.pitch, 'roll': cam.roll}
        # Transform points to camera CRS on GPU
        cam_x, cam_y, cam_z = transform_to_camera_crs(x, y, z, cam_ori)
            
        u, v = vmap(spherical_projection, in_axes=(0, 0, 0, None, None))(cam_x, cam_y, cam_z, width, height)

        # Handle wrapping around the image edges
        if u.max() - u.min() > width / 2:
            # If the range of u exceeds half the image width, keep the largest segment
            u_right = jnp.where(u > width / 2, u , width-1)  # Wrap around
            u_left = jnp.where(u < width / 2, u , 0)  # Wrap around
            # compare the two segments and keep the one with the larger range
            u = u_right if jnp.ptp(u_right) > jnp.ptp(u_left) else u_left

        u = jnp.clip(u, 0, width - 1)  # Ensure u is within image bounds
        i, j = u.astype(int), v.astype(int)
        
        # create a polygon with image coordinates and save to COCO format 
        # Create a polygon from the image coordinates
        coords = np.array([i, j]).T
        img_polygon = Polygon(coords)
        # Calculate the area
        area = img_polygon.area

        # Retrieve the bounding box coordinates
        minx, miny, maxx, maxy = img_polygon.bounds

        # Compute width and height
        w = maxx - minx
        h = maxy - miny

        if w >= 1000:
            # Skip if the bounding box is too large
            continue
        # Format the bounding box for COCO: [x, y, width, height]
        bbox = [float(minx), float(miny), float(w), float(h)]

        # Append to annotations list    
        annotations.append({
            "id": annotation_id,
            "object_id": gt_idx,
            "image_id": cam_idx,
            "category_id": 1,
            "segmentation": [list(coords.flatten().astype('float'))],
            "area": area,
            "bbox": bbox,
            "iscrowd": 0
        })
        annotation_id += 1

    # Define image metadata
    image_info_ls.append({
        "id": cam_idx,
        "file_name": cam.File,
        "width": width,
        "height": height
    })

# Define categories
categories = [
    {
        "id": 1,
        "name": "manhole",
        "supercategory": "none"
    }
]

# Compile the final COCO structure
coco_format = {
    "images": image_info_ls,
    "annotations": annotations,
    "categories": categories
}

# Save to a JSON file
with open("ne_steep_recalc_COCO.json", "w") as json_file:
    json.dump(coco_format, json_file, indent=4)


138it [00:02, 48.98it/s]


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Polygon as plt_Polygon
from PIL import Image
import numpy as np

# Load the image
image_path = '20200408_073125_003877.jpg'  # Replace with your image path
image = Image.open(image_path)


polygon_coords = list(coords)
# Convert to NumPy array for processing
polygon_array = np.array(coords)

# Calculate bounding box: (min_x, min_y, width, height)
min_x = np.min(polygon_array[:, 0])
min_y = np.min(polygon_array[:, 1])
width = np.max(polygon_array[:, 0]) - min_x
height = np.max(polygon_array[:, 1]) - min_y

# Create a figure and axis
fig, ax = plt.subplots(figsize=(20, 20))

# Display the image
ax.imshow(image)
# Create and add the polygon patch
polygon_patch = plt_Polygon(polygon_coords, closed=True, edgecolor='blue', facecolor='blue', alpha=0.5)
ax.add_patch(polygon_patch)
# Create and add the bounding box patch
bbox_patch = Rectangle((min_x, min_y), width, height, linewidth=2, edgecolor='red', facecolor='none')
ax.add_patch(bbox_patch)

# Optional: Add labels
ax.text(min_x, min_y - 10, 'Polygon', color='blue', fontsize=12)
ax.text(min_x, min_y - 25, 'Bounding Box', color='red', fontsize=12)

# Set plot limits and show the plot
ax.set_xlim(0, image.width)
ax.set_ylim(image.height, 0)  # Invert y-axis to match image coordinates
plt.axis('off')  # Hide axes
plt.show()


In [43]:
min_x, min_y, width,height

(4472, 2676, 225, 109)

# Experimental Code

In [5]:
import pyproj
import numpy as np

def get_grid_convergence_angle(lon, lat, delta_lat=1e-5):
    """
    Estimate grid convergence angle (true north - grid north) in degrees.
    Positive if grid north is east of true north.
    """
    transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
    x1, y1 = transformer.transform(lon, lat)
    x2, y2 = transformer.transform(lon, lat + delta_lat)
    dx = x2 - x1
    dy = y2 - y1
    angle_rad = np.arctan2(dx, dy)
    return np.rad2deg(angle_rad)

def correct_yaw_for_lv95(yaw_true_north, lon, lat):
    convergence = get_grid_convergence_angle(lon, lat)
    yaw_lv95 = yaw_true_north - convergence
    return yaw_lv95, convergence


# WGS84 input: GPS + ENU orientation
lon = 6.88926116691478       # Lausanne
lat = 46.98733611884436
height = 545.34600000000000    # meters above ellipsoid
yaw = 235.61471396512013        # Facing East in ENU
pitch = -2.37668345474374
roll = 0.55190434543908



yaw_lv95, convergence = correct_yaw_for_lv95(yaw, lon, lat)

print(f"Corrected Yaw (LV95): {yaw_lv95:.6f}°")
print(f"Grid Convergence Angle: {convergence:.6f}°")



Corrected Yaw (LV95): 235.212548°
Grid Convergence Angle: 0.402166°
